In [1]:
import requests
import pandas as pd

# Login

In [2]:
url = "http://127.0.0.1:9000/api/".replace("\\","/")

In [3]:
username="zeladorx@zeladorx.com.br"
password="BETpet1516%"

body = {
    "username":username,
    "password":password
}

In [4]:
response = requests.post("http://127.0.0.1:9000/api/login", body)

In [5]:
response.json()

{'token': '92ca37bb8f53bb9a8fbaefad1155c6ab55677c11',
 'id_random': '6uW6BnjPFHyd',
 'user_id': 1,
 'username': 'zeladorx',
 'email': 'zeladorx@zeladorx.com.br',
 'is_superuser': True}

# Auth

In [6]:
headers = {
    "Authorization": f"Token {response.json()['token']}"
}

# Servicos agendados

In [7]:
# retorna todas as servicos cadastradas, que o usário tem permissão para visualizar
# codigo de permissão necessário --> 322: Pode visualizar serviços agendados

response = requests.get("http://127.0.0.1:9000/api/servicos/agendados/ListServicoLimpezaPredialAgendadoAnotados/", headers=headers)
response.json()['results']
df = pd.DataFrame(response.json()['results'])
display(df)

,id,id_random,DescricaoDoServico,DataDeInicio,DataDeConclusao,TipoServico,Areas,ServicosEscalados,dias_diferenca,novo_status
0,1,UWoNKAzEeZRs,steste,2025-07-10T18:04:00-03:00,2025-07-10T18:04:00-03:00,Regular,1,[1],-38,Atrasado


## Criar nova servicos agendados

In [9]:
## As colunas, Areas, ServicosEscalados, são chaves estrangeiras e precisam estar devidamente configuradas
## sendo obriagatorios

In [11]:
# lista de areas dispniveis para solitarem acesso
response = requests.get("http://127.0.0.1:9000/api/areas/ListAreasLimpezaPredialFromForms/", headers=headers)
response.json()['results']
df = pd.DataFrame(response.json()['results'])
display(df)
print()

# lista de areas dispniveis para solitarem acesso
response = requests.get("http://127.0.0.1:9000/api/catalogo_de_servicos/ListCatalogodeServicoLimpezaPredialFromForms/", headers=headers)
response.json()['results']
df = pd.DataFrame(response.json()['results'])
display(df)
print()

,id,id_random,nome,dimensao,servico,localidade,foto,status
0,1,3SEb3VkCfd17,teste 2,1000.0,1,1,None,Mobilizado
1,2,xJvj4BokKSE0,Atest,263.0,1,1,None,Mobilizado


,id,id_random,nome,EmpresaSecundaria,status
0,1,jnGZ7u6GDeL2,Empresa primaria de teste jajaalsdklasjdklsajd...,1,Mobilizado


In [13]:
# exemplo de envio de boby para criar uma no servico
body = {
    'Areas': 1, # vem de api/areas/ListAreasLimpezaPredialFromForms/
    'TipoServico': "Regular", #opcoes validas Regular, Extra, Automático
    'DescricaoDoServico':  "teste jupyter agndamento", 
    'ServicosEscalados': 1, # vem de api/catalogo_de_servicos/ListCatalogodeServicoLimpezaPredialFromForms/
    'DataDeInicio': "2025-08-08T13:57:00-03:00", #agendamento para 11/08/2025 13:57 -- > ajusta o fuso horario :00-03:00
    'DataDeConclusao': "2025-08-11T19:57:00-03:00",
    'foto_solicitacao': "", # file com uma foto da area no momento da solicitação
    'foto_entrega': "", # file com uma foto da area no momento da entrega
}

In [17]:
# codigo de permissão necessário --> 320: Pode agendar novos serviços
response = requests.post("http://127.0.0.1:9000/api/servicos/agendados/CreateServicosLimpezaPredial/", body, headers=headers)
response.json()

{'success': True,
 'message': 'ServicoLimpezaPredialAgendado criado com sucesso.',
 'data': {'id': 2,
  'id_random': 'ghfac2GGmKZY',
  'Areas': 1,
  'TipoServico': 'Regular',
  'DescricaoDoServico': 'teste jupyter agndamento',
  'ServicosEscalados': [1],
  'DataDeInicio': '2025-08-08T13:57:00-03:00',
  'DataDeConclusao': '2025-08-11T19:57:00-03:00'}}

# Detalhes da serviço agendado

In [18]:
# enviar o id random do servico que quer se avaliar
# codigo de permissao 322: Pode visualizar serviços agendados
response = requests.get("http://127.0.0.1:9000/api/servicos/agendados/ServicosLimpezaPredialDetail/ghfac2GGmKZY/", headers=headers)
response.json()

{'id': 2,
 'id_random': 'ghfac2GGmKZY',
 'Areas': 1,
 'TipoServico': 'Regular',
 'DescricaoDoServico': 'teste jupyter agndamento',
 'ServicosEscalados': [1],
 'DataDeInicio': '2025-08-08T13:57:00-03:00',
 'DataDeConclusao': '2025-08-11T19:57:00-03:00'}

# atualizar servico agendado

In [19]:
# as chaves body da atualização é exatamente igual o body da criacao
# atualiando a periodicidade para bimestral

body = {
    'Areas': 1, # vem de api/areas/ListAreasLimpezaPredialFromFoms/
    'TipoServico': "Regular", #opcoes validas Regular, Extra, Automático
    'DescricaoDoServico':  "teste jupyter agndamento", 
    'ServicosEscalados': 1, # vem de api/catalogo_de_servicos/ListCatalogodeServicoLimpezaPredialFromForms/
    'DataDeInicio': "2025-08-08T13:57:00-03:00", #agendamento para 11/08/2025 13:57 -- > ajusta o fuso horario :00-03:00
    'DataDeConclusao': "2025-08-11T19:57:00-03:00",
    'foto_solicitacao': "", # file com uma foto da area no momento da solicitação
    'foto_entrega': "", # file com uma foto da area no momento da entrega
}

In [20]:
# essa rota recebe o id random da servico como parametro de url, consultar em api/servicos/agendados/ServicosLimpezaPredialUpdate/id_random/
# atualizando a área criada anteriormente, obeserve o id_random
# codigo de permissão necessário --> 321: Pode editar serviços agendados

response = requests.put("http://127.0.0.1:9000/api/servicos/agendados/ServicosLimpezaPredailUpdate/ghfac2GGmKZY/", body, headers=headers)
response.json()

{'success': True,
 'message': 'ServicoLimpezaPredialAgendado atualizado com sucesso.',
 'data': {'id': 2,
  'id_random': 'ghfac2GGmKZY',
  'Areas': 1,
  'TipoServico': 'Regular',
  'DescricaoDoServico': 'teste jupyter agndamento',
  'ServicosEscalados': [1],
  'DataDeInicio': '2025-08-08T13:57:00-03:00',
  'DataDeConclusao': '2025-08-11T19:57:00-03:00'}}

# Alterar Status do servico

In [22]:
# essa rota recebe o id random da servico como parametro de url, consultar em servicos/agendados/ListServicoLimpezaPredialAgendadoAnotados/
# atualizando a servico criada anteriormente, obeserve o id_random
# codigo de permissão necessário --> 
# 320: Pode agendar novos serviços para Agendado (Desmobilizado) ou
# 361: Pode acompanhar serviços agendados para si próprio ou 324: Pode acompanhar serviços agendados para Em andamento (Mobilizado)
# 326: Pode concluir serviços em andamento para Concluido
# 328: Pode cancelar serviços agendados para Cancelado


STATUS_PERMISSION_MAP = {
    'Agendado': ['320: Pode agendar novos serviços'],
    'Em andamento': [
        '361: Pode acompanhar serviços agendados para si próprio',
        '324: Pode acompanhar serviços agendados'
    ],
    'Concluido': ['326: Pode concluir serviços em andamento'],
    'Cancelado': ['328: Pode cancelar serviços agendados']
}


body = {
    'status': 'Em andamento', # pde ser Agendado, Em andamento, Concluido, Cancelado
}

response = requests.put("http://127.0.0.1:9000/api/servicos/agendados/AlterStatusServicoLimpezaPredial/ghfac2GGmKZY/", body, headers=headers)
response.json()

{'detail': "Status alterado com sucesso para 'Em andamento'."}

# Se deletar Serviço

In [23]:
# essa rota recebe o id random da servico como parametro de url, consultar em api/areas/ListAreasLimpezaPredial/
# codigo de permissão necessário --> 323: Pode excluir serviços agendados

response = requests.get("http://127.0.0.1:9000/api/servicos/agendados/IfDeleteServicoAgendadoLimpezaPredial/ghfac2GGmKZY/", headers=headers)
response.json()

{'objeto_principal': 'teste 2 | ççaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa -- unidade jupyter | Empresa primaria de teste jajaalsdklasjdklsajdkasjdklasjkdlkasdlkasjdlkasjkdlkas | 08/08/2025 16:57',
 'resumo': {'ServicoLimpezaPredialAgendado': 1,
  'ServicoLimpezaPredialAgendado_ServicosEscalados': 1},
 'objetos_em_cascata': ['- teste 2 | ççaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa -- unidade jupyter | Empresa primaria de teste jajaalsdklasjdklsajdkasjdklasjkdlkasdlkasjdlkasjkdlkas | 08/08/2025 16:57',
  '  - ServicoLimpezaPredialAgendado_ServicosEscalados object (2)']}

# Deleção de serviços

In [26]:
# essa rota recebe o id random da servico como parametro de url, consultar em api/areas/ListAreasLimpezaPredial/
# codigo de permissão necessário --> 323: Pode excluir serviços agendados

response = requests.delete("http://127.0.0.1:9000/api/servicos/agendados/DeleteServicoAgendadoLimpezaPredial/ghfac2GGmKZY/", headers=headers)
response.json()

{'detail': "ServicoLimpezaPredialAgendado 'teste 2 | ççaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa -- unidade jupyter | Empresa primaria de teste jajaalsdklasjdklsajdkasjdklasjkdlkasdlkasjdlkasjkdlkas | 08/08/2025 16:57' foi deletado com sucesso."}